In [ ]:
# Importamos las librerías necesarias
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:


# Creamos una sesión de Spark
spark = SparkSession.builder.appName("GBTClassifier").getOrCreate()


In [ ]:

# Cargamos el conjunto de datos (usaremos el conjunto de datos "Iris" de UCI)
data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")


In [ ]:

# Seleccionamos las columnas necesarias y las renombramos
data = data.selectExpr("_c0 as sepal_length", "_c1 as sepal_width", "_c2 as petal_length", "_c3 as petal_width", "_c4 as species")


In [ ]:

# Convertimos las variables de entrada en un vector de características
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
data = assembler.transform(data)


In [ ]:

# Convertimos la variable objetivo en un índice numérico
labelIndexer = StringIndexer(inputCol="species", outputCol="label")
data = labelIndexer.fit(data).transform(data)


In [ ]:

# Dividimos el conjunto de datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=123)


In [ ]:

# Creamos un modelo de Gradient Boosted Trees
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)


In [ ]:

# Entrenamos el modelo con el conjunto de entrenamiento
model = gbt.fit(trainingData)


In [ ]:

# Obtenemos las predicciones con el conjunto de prueba
predictions = model.transform(testData)


In [ ]:

# Evaluamos la precisión del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)


In [ ]:

# Imprimimos la precisión del modelo
print("GBTClassifier - Precisión: {:.2f}%".format(accuracy*100))


In [ ]:

# Detenemos la sesión de Spark
spark.stop()